**What is RFM Analysis?**

RFM stands for Recency, Frequency, and Monetary value, each corresponding to some key customer trait. 
These RFM metrics are important indicators of a customer’s behavior because frequency and monetary value affects a customer’s lifetime value, and recency affects retention, a measure of engagement.

RFM analysis helps marketers find answers to the following questions:

    Who are your best customers?
    Which of your customers could contribute to your churn rate?
    Who has the potential to become valuable customers?
    Which of your customers can be retained?
    Which of your customers are most likely to respond to engagement campaigns?

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pure_placedOrders_new.csv')

In [3]:
df.head(3)

,InvoiceNo,StockCode,Description,InvoiceDate,UnitPrice,CustomerID,Country,FinalQuantity
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,2.55,17850,United Kingdom,6
1,536365,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,3.39,17850,United Kingdom,6
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,2.75,17850,United Kingdom,8


In [4]:
df.Country.nunique()

37

In [5]:
customer_country=df[['Country','CustomerID']].drop_duplicates()

x = customer_country.groupby(['Country'])['CustomerID'].aggregate('count')/customer_country.shape[0] * 100
x.reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
35,United Kingdom,90.235457
14,Germany,2.169898
13,France,2.008310
30,Spain,0.669437
3,Belgium,0.577101
32,Switzerland,0.484765
26,Portugal,0.438596
18,Italy,0.323176
12,Finland,0.277008
1,Austria,0.253924


In [6]:
# df = df.loc[df['Country'] == 'United Kingdom']

In [8]:
df.head(3)

,InvoiceNo,StockCode,Description,InvoiceDate,UnitPrice,CustomerID,Country,FinalQuantity
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,2.55,17850,United Kingdom,6
1,536365,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,3.39,17850,United Kingdom,6
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,2.75,17850,United Kingdom,8


In [9]:
df.isnull().sum(axis=0)

InvoiceNo        0
StockCode        0
Description      0
InvoiceDate      0
UnitPrice        0
CustomerID       0
Country          0
FinalQuantity    0
dtype: int64

In [10]:
df = df[df['CustomerID'].isnull() == False] #pd.notnull
df.isnull().sum(axis=0)

InvoiceNo        0
StockCode        0
Description      0
InvoiceDate      0
UnitPrice        0
CustomerID       0
Country          0
FinalQuantity    0
dtype: int64

In [11]:
df.describe()

,InvoiceNo,UnitPrice,CustomerID,FinalQuantity
count,383873.000000,383873.000000,383873.000000,383873.000000
mean,560580.663584,2.867834,15290.860550,12.764367
std,13069.627045,4.135378,1710.708868,42.227435
min,536365.000000,0.040000,12347.000000,1.000000
25%,549245.000000,1.250000,13969.000000,2.000000
50%,561869.000000,1.950000,15152.000000,6.000000
75%,572037.000000,3.750000,16791.000000,12.000000
max,581587.000000,649.500000,18287.000000,4800.000000


In [12]:
df.shape

(383873, 8)

In [13]:
df.nunique()

InvoiceNo        18339
StockCode         3645
Description       3627
InvoiceDate      17111
UnitPrice          380
CustomerID        4324
Country             37
FinalQuantity      328
dtype: int64

Adding a column for total price

In [14]:
df['TotalPrice'] = df['FinalQuantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,InvoiceDate,UnitPrice,CustomerID,Country,FinalQuantity,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,2.55,17850,United Kingdom,6,15.30
1,536365,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,3.39,17850,United Kingdom,6,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,2.75,17850,United Kingdom,8,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,3.39,17850,United Kingdom,6,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,3.39,17850,United Kingdom,6,20.34


Check the min and max values in Unit price column

In [15]:
df['InvoiceDate'].min()

'2010-12-01 08:26:00'

In [16]:
df['InvoiceDate'].max()

'2011-12-09 12:50:00'

Since recency is calculated for a point in time. The last invoice date is 2011-12-09, this is the date we will use to calculate recency.

In [17]:
import datetime as dt
NOW = dt.datetime(2011,12,10)

In [18]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

Creating a RFM table

In [74]:
rfmTable = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                        'InvoiceNo': lambda x: len(x),      # Frequency
                                        'TotalPrice': lambda x: x.sum()}) # Monetary Value

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)

In [75]:
rfmTable.columns

Index(['InvoiceDate', 'InvoiceNo', 'TotalPrice'], dtype='object')

In [76]:
rfmTable.columns = ['recency', 'frequency' , 'monetary_value']

Calculate RFM metrics for each customer

In [77]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12347,2,182,4310.00
12348,75,23,1462.20
12349,18,72,1457.55
12350,310,16,294.40
12352,36,70,1265.41


In [78]:
(NOW - dt.datetime(2011,1,18)).days==326

True

The easiest way to split metrics into segments is by using quartile. 

1. This gives us a starting point for detailed analysis
2. 4 segments are easy to understand and explain

In [79]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,17.0,17.0,299.665
0.50,50.0,40.0,653.270
0.75,142.0,97.0,1611.890


In [80]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0},
 'frequency': {0.25: 17.0, 0.5: 40.0, 0.75: 97.0},
 'monetary_value': {0.25: 299.66499999999996,
  0.5: 653.27,
  0.75: 1611.8899999999999}}

Creating segmented RFM Table

In [81]:
segmented_rfm = rfmTable

Lowest recency, highest frequency and monetary are our best customers 

In [82]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [87]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

17.0
142.0
50.0
50.0
50.0
50.0
17.0
142.0
142.0
17.0
142.0
17.0
17.0
142.0
142.0
142.0
50.0
17.0
142.0
142.0
50.0
17.0
50.0
17.0
142.0
50.0
142.0
142.0
50.0
50.0
50.0
142.0
50.0
50.0
50.0
50.0
142.0
142.0
142.0
50.0
17.0
142.0
142.0
17.0
142.0
17.0
142.0
50.0
50.0
17.0
50.0
50.0
50.0
17.0
142.0
142.0
142.0
17.0
17.0
17.0
50.0
50.0
142.0
50.0
50.0
17.0
17.0
142.0
142.0
50.0
142.0
142.0
50.0
142.0
17.0
17.0
17.0
50.0
142.0
17.0
142.0
17.0
50.0
17.0
17.0
50.0
50.0
17.0
50.0
17.0
17.0
50.0
142.0
17.0
142.0
50.0
50.0
142.0
50.0
142.0
50.0
142.0
142.0
142.0
142.0
50.0
17.0
142.0
142.0
142.0
50.0
50.0
17.0
17.0
142.0
17.0
142.0
17.0
50.0
142.0
142.0
142.0
50.0
50.0
50.0
50.0
50.0
142.0
50.0
142.0
50.0
17.0
50.0
17.0
17.0
17.0
142.0
50.0
17.0
50.0
17.0
50.0
50.0
50.0
142.0
50.0
17.0
17.0
17.0
17.0
50.0
50.0
142.0
142.0
50.0
142.0
50.0
17.0
50.0
50.0
142.0
50.0
142.0
142.0
50.0
142.0
142.0
142.0
17.0
142.0
50.0
50.0
17.0
17.0
17.0
142.0
50.0
17.0
17.0
17.0
142.0
142.0
142.0
142.0
50.0
50.0
142.

50.0
50.0
50.0
50.0
50.0
50.0
50.0
142.0
142.0
17.0
50.0
142.0
142.0
17.0
50.0
17.0
142.0
142.0
50.0
50.0
50.0
50.0
142.0
50.0
142.0
142.0
50.0
50.0
142.0
17.0
50.0
50.0
17.0
142.0
50.0
17.0
142.0
17.0
17.0
50.0
50.0
17.0
17.0
17.0
142.0
17.0
142.0
17.0
142.0
17.0
142.0
50.0
17.0
50.0
142.0
142.0
142.0
17.0
142.0
142.0
50.0
50.0
142.0
142.0
142.0
50.0
50.0
17.0
50.0
17.0
17.0
50.0
17.0
142.0
17.0
50.0
17.0
17.0
142.0
142.0
142.0
50.0
50.0
17.0
50.0
142.0
17.0
17.0
17.0
17.0
50.0
142.0
142.0
17.0
142.0
50.0
50.0
17.0
17.0
142.0
17.0
17.0
50.0
142.0
50.0
50.0
17.0
17.0
50.0
17.0
17.0
50.0
142.0
17.0
50.0
50.0
142.0
50.0
17.0
17.0
142.0
17.0
50.0
17.0
17.0
50.0
142.0
17.0
50.0
50.0
142.0
142.0
17.0
50.0
17.0
17.0
17.0
17.0
17.0
50.0
142.0
50.0
50.0
17.0
50.0
17.0
50.0
50.0
17.0
142.0
17.0
142.0
50.0
50.0
142.0
17.0
17.0
50.0
50.0
17.0
17.0
17.0
17.0
142.0
17.0
50.0
142.0
50.0
142.0
17.0
17.0
17.0
17.0
50.0
142.0
142.0
17.0
142.0
142.0
17.0
17.0
17.0
142.0
17.0
17.0
17.0
142.0
142.0
50.0
1

In [28]:
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12347,2,182,4310.00,1,1,1
12348,75,23,1462.20,3,3,2
12349,18,72,1457.55,2,2,2
12350,310,16,294.40,4,4,4
12352,36,70,1265.41,2,2,2


RFM segments split your customer base into an imaginary 3D cube. It is hard to visualize. However, we can sort it out.

Add a new column to combine RFM score, 111 is the highest score as we determined earlier.

In [29]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12347,2,182,4310.00,1,1,1,111
12348,75,23,1462.20,3,3,2,332
12349,18,72,1457.55,2,2,2,222
12350,310,16,294.40,4,4,4,444
12352,36,70,1265.41,2,2,2,222


In [32]:
segmented_rfm.RFMScore.unique()

array(['111', '332', '222', '444', '422', '443', '221', '211', '142',
       '311', '333', '122', '433', '144', '322', '233', '143', '421',
       '223', '243', '123', '431', '121', '244', '331', '343', '133',
       '321', '132', '334', '342', '411', '434', '344', '213', '131',
       '312', '134', '412', '232', '313', '441', '432', '442', '141',
       '341', '231', '212', '324', '423', '112', '234', '113', '242',
       '424', '323', '224', '124', '241', '413', '114'], dtype=object)

**RFM factors illustrate these facts:**

1. the more recent the purchase, the more responsive the customer is to promotions
2. the more frequently the customer buys, the more engaged and satisfied they are
3. monetary value differentiates heavy spenders from low-value purchasers

### These are our most valuable customers

In [69]:
print("Total number of most valuable customers: ", len(segmented_rfm[segmented_rfm['RFMScore']=='111']))
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)


Total number of most valuable customers:  447


,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
14646,1,2058,278778.02,1,1,1,111
18102,0,431,259657.30,1,1,1,111
17450,8,334,189575.43,1,1,1,111
14911,1,5487,129560.52,1,1,1,111
14156,9,1374,113733.70,1,1,1,111
17511,2,950,88479.55,1,1,1,111
16684,4,274,65920.12,1,1,1,111
13694,3,560,63113.22,1,1,1,111
15311,0,2338,59645.82,1,1,1,111


based on Discount, expenditure, quantity, no of objects

### Potential most valuable customers


Our recent customers with average frequency and who spent a good amount.
We can offer membership or loyalty programs or recommend related products to upsell them and help them become your Loyal customers or most valuable customers.

In [45]:
segmented_rfm[segmented_rfm['RFMScore']=='121'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
17949,1,63,53167.74,1,2,1,121
16333,7,45,26626.80,1,2,1,121
17857,4,48,25185.84,1,2,1,121
14866,10,96,14160.95,1,2,1,121
14607,15,52,10221.10,1,2,1,121
18092,2,89,9065.76,1,2,1,121
17306,10,66,8600.73,1,2,1,121
12989,3,63,6860.81,1,2,1,121
15482,15,72,6799.92,1,2,1,121


In [46]:
segmented_rfm[segmented_rfm['RFMScore']=='131'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12557,16,33,11990.96,1,3,1,131
13225,3,31,6083.04,1,3,1,131
17133,15,27,4892.24,1,3,1,131
12664,8,28,4391.88,1,3,1,131
15971,17,34,4194.90,1,3,1,131
13953,7,20,3515.68,1,3,1,131
16107,17,31,3204.16,1,3,1,131
13868,7,33,3192.54,1,3,1,131
13685,2,39,3119.44,1,3,1,131


In [47]:
segmented_rfm[segmented_rfm['RFMScore']=='122'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16496,8,93,1562.22,1,2,2,122
15117,14,79,1551.78,1,2,2,122
15611,9,41,1547.39,1,2,2,122
16365,17,51,1540.42,1,2,2,122
16790,3,93,1532.87,1,2,2,122
16748,17,69,1515.90,1,2,2,122
15392,4,84,1515.73,1,2,2,122
16817,2,84,1496.25,1,2,2,122
12826,2,91,1468.12,1,2,2,122


In [49]:
segmented_rfm[segmented_rfm['RFMScore']=='131'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12557,16,33,11990.96,1,3,1,131
13225,3,31,6083.04,1,3,1,131
17133,15,27,4892.24,1,3,1,131
12664,8,28,4391.88,1,3,1,131
15971,17,34,4194.90,1,3,1,131
13953,7,20,3515.68,1,3,1,131
16107,17,31,3204.16,1,3,1,131
13868,7,33,3192.54,1,3,1,131
13685,2,39,3119.44,1,3,1,131


In [50]:
segmented_rfm[segmented_rfm['RFMScore']=='141'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16000,2,9,12393.70,1,4,1,141
12798,10,8,6977.04,1,4,1,141
16532,17,4,6748.80,1,4,1,141
15195,2,1,3861.00,1,4,1,141


### At risk Customers

Customers whose monetary value and frequency is more but recency is very less, are the customers we might loose so to gain them we can give them personalised coupons on their most frequent purchase or other benefits to encourage them to shop more 

In [51]:
segmented_rfm[segmented_rfm['RFMScore']=='411'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13093,275,159,7797.58,4,1,1,411
17850,372,295,5349.34,4,1,1,411
14016,161,160,4278.46,4,1,1,411
15808,306,191,3648.72,4,1,1,411
15379,169,193,3631.89,4,1,1,411
13952,217,136,3246.09,4,1,1,411
17504,206,115,2745.28,4,1,1,411
12840,143,113,2715.92,4,1,1,411
16919,156,326,2592.25,4,1,1,411


In [52]:
segmented_rfm[segmented_rfm['RFMScore']=='311'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12409,78,109,11061.88,3,1,1,311
16180,100,162,10240.43,3,1,1,311
12744,56,215,9120.39,3,1,1,311
14952,59,138,8042.09,3,1,1,311
16745,86,351,7157.10,3,1,1,311
16652,58,117,6683.32,3,1,1,311
12359,57,242,6140.48,3,1,1,311
17509,57,364,6070.49,3,1,1,311
12637,67,374,5790.25,3,1,1,311


In [53]:
segmented_rfm[segmented_rfm['RFMScore']=='211'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12415,24,711,123658.45,2,1,1,211
16029,38,227,63819.81,2,1,1,211
14680,25,254,27194.93,2,1,1,211
12753,22,196,21065.85,2,1,1,211
12731,23,271,18660.41,2,1,1,211
12678,42,156,16291.26,2,1,1,211
15513,33,306,14518.88,2,1,1,211
12477,22,298,13127.76,2,1,1,211
17340,29,407,12225.69,2,1,1,211


In [38]:
segmented_rfm[segmented_rfm['RFMScore']=='221'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12931,21,78,35259.81,2,2,1,221
16656,22,72,8578.18,2,2,1,221
13629,23,59,7792.36,2,2,1,221
15465,19,97,7266.17,2,2,1,221
13324,36,75,7072.80,2,2,1,221
13316,37,86,5732.93,2,2,1,221
13941,21,45,4912.41,2,2,1,221
12500,23,50,4024.17,2,2,1,221
15129,50,84,3447.40,2,2,1,221


### Customers we can't loose 

These customers have low recency, frequency and monetary value, i.e they used to visit us but not now, so we can provide them relevent coupons or run a survey to find what went wrong and can try multiple marketing schemes to revive them

In [56]:
segmented_rfm[segmented_rfm['RFMScore']=='411'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13093,275,159,7797.58,4,1,1,411
17850,372,295,5349.34,4,1,1,411
14016,161,160,4278.46,4,1,1,411
15808,306,191,3648.72,4,1,1,411
15379,169,193,3631.89,4,1,1,411
13952,217,136,3246.09,4,1,1,411
17504,206,115,2745.28,4,1,1,411
12840,143,113,2715.92,4,1,1,411
16919,156,326,2592.25,4,1,1,411


In [54]:
segmented_rfm[segmented_rfm['RFMScore']=='431'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12980,157,20,7161.96,4,3,1,431
14439,319,32,2661.24,4,3,1,431
14459,155,37,1837.92,4,3,1,431
16152,268,29,1829.04,4,3,1,431
12424,162,30,1760.96,4,3,1,431
15332,366,35,1661.06,4,3,1,431


In [55]:
segmented_rfm[segmented_rfm['RFMScore']=='421'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12590,211,67,9338.38,4,2,1,421
16553,163,83,5668.82,4,2,1,421
15032,256,48,4464.10,4,2,1,421
17230,264,67,3470.22,4,2,1,421
12754,235,63,3010.74,4,2,1,421
12625,211,83,2367.42,4,2,1,421
12653,148,43,2209.85,4,2,1,421
17597,213,55,2044.37,4,2,1,421
12585,234,85,1768.51,4,2,1,421


#### can be a new customer so need some attention from marketing

In [41]:
segmented_rfm[segmented_rfm['RFMScore']=='141'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16000,2,9,12393.70,1,4,1,141
12798,10,8,6977.04,1,4,1,141
16532,17,4,6748.80,1,4,1,141
15195,2,1,3861.00,1,4,1,141


In [34]:
segmented_rfm.to_csv('segmented_OnlineRetail.csv',index=False)